# **Klasifikasi Room Type berdasarkan Price, Review, Minimum Night, dan Availability**

**Ahmad Habib Fitriansyah || 1301174056 || IF-41-12 || Air Bnb**

# Formula Permasalahan

Pada tabel air bnb terdapat kolom **price, review, minimum night, dan juga availability**. Untuk melalukan klasifikasi room type, saya menggunakan kolom-kolom tersebut. Metode yang digunakan adalah **Naive Bayes.**

# Eksplorasi dan Persiapan Data

Hal yang dilakukan adalah misahkan setiap kolom **price, review, minimum night, dan juga availability** dan dimasukan ke dalam list yang berbeda. Hal ini dilakukan untuk mempermudah melakukan persiapan data.

Selain itu, ada beberapa hal lagi yang dilakukan antara lain :


1.   Menghapus data kosong
2.   Menghapus data yang terdeteksi sebagai outlier
3.   Mengubah label / data target menjadi angka contohnya Private Room = 1 dan lain - lain

# Pemodelan

# **Import**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import csv
import math
import statistics
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split

# Ahmad Habib Fitriansyah
# Metro, Indonesia
# Tuesday, 03 March 2020, 20:01 PM

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Read Dataset**

In [0]:
def readDataset():
  price = []
  review = []
  minimumNight = []
  availability = []
  roomType = []
  a = [] # PRICE
  b = [] # MINIMUM NIGHT
  c = [] # REVIEW
  d = [] # AVAILABILITY
  e = [] # ROOM TYPE

  # Case 1 = price, minimum night, review per month, availability

  with open('/content/drive/My Drive/Machine Learning/air_bnb.csv', mode='r') as csv_input:
        data = csv.reader(csv_input)
        next(data)
        for row in data:
            a.append(row[9])
            b.append(row[10])
            c.append(row[13])
            d.append(row[15])
            e.append(row[8])

  # Memasukan tabel price ke list bertujuan untuk memisahkan antar kolom yg ingin diambil
  for i in range(len(a)):
    price.append(int(a[i]))

  # Memasukan tabel minimum night ke list bertujuan untuk memisahkan antar kolom yg ingin diambil
  for j in range(len(b)):
    minimumNight.append(int(b[j]))

  # Memasukan tabel review ke list bertujuan untuk memisahkan antar kolom yg ingin diambil
  for k in range(len(c)):
    review.append(c[k])

  # Memasukan tabel availability ke list bertujuan untuk memisahkan antar kolom yg ingin diambil
  for l in range(len(d)):
    availability.append(int(d[l]))

  # Memasukan tabel room type ke list bertujuan untuk memisahkan antar kolom yg ingin diambil
  for m in range(len(e)):
    roomType.append(e[m])

  return price, minimumNight, review, availability, roomType

**Outlier**

In [0]:
def outlier(price, minNight, review, availability, roomType):

  # Kodingan ini digunakan untuk mendeteksi outlier. Jika sebuah data melebihi outliernya, maka data tersebut dihapus

  an_array_price = np.array(price) # memasukan data price kedalam array
  mean = np.mean(an_array_price) 
  standard_deviation = np.std(an_array_price)
  distance_from_mean = abs(an_array_price - mean)
  max_deviations = 2
  not_outlier_price = distance_from_mean < max_deviations * standard_deviation

  an_array_minNight = np.array(minNight)
  mean = np.mean(an_array_minNight)
  standard_deviation = np.std(an_array_minNight)
  distance_from_mean = abs(an_array_minNight - mean)
  max_deviations = 2
  not_outlier_minNight = distance_from_mean < max_deviations * standard_deviation

  an_array_review = np.array(review)
  mean = np.mean(an_array_review)
  standard_deviation = np.std(an_array_review)
  distance_from_mean = abs(an_array_review - mean)
  max_deviations = 2
  not_outlier_review = distance_from_mean < max_deviations * standard_deviation

  an_array_availa = np.array(availability)
  mean = np.mean(an_array_availa)
  standard_deviation = np.std(an_array_availa)
  distance_from_mean = abs(an_array_availa - mean)
  max_deviations = 2
  not_outlier_availa = distance_from_mean < max_deviations * standard_deviation

  an_array_roomType = np.array(roomType)
    
  no_outliers_review = []
  no_outliers_minNight = []
  no_outliers_price = []
  no_outliers_availa = []
  no_outliers_roomType = []

  for nn in range(len(review)):
    if (not_outlier_price[nn] == True) and (not_outlier_review[nn] == True) and (not_outlier_minNight[nn] == True) and (not_outlier_availa[nn] == True):
      no_outliers_review.append(an_array_review[nn])
      no_outliers_price.append(an_array_price[nn])
      no_outliers_minNight.append(an_array_minNight[nn])
      no_outliers_availa.append(an_array_availa[nn])
      no_outliers_roomType.append(roomType[nn])

  return no_outliers_price, no_outliers_minNight, no_outliers_review, no_outliers_availa, no_outliers_roomType

**Scaling : Using Min-Max Normalization to 0-1**

In [0]:
def scalingData(data):

  minValue = 9999999
  maxValue = 0
  newData = []

    # Find Max Value
  for i in range(len(data)):
    if maxValue < data[i]:
      maxValue = data[i]
    
    # Find Min Value
  for j in range(len(data)):
    if minValue > data[j]:
      minValue = data[j]

    # Min-Max Normalization
  for k in range(len(data)):
    temp = (data[k] - minValue) / (maxValue - minValue)
    newData.append(temp)

  return newData


**Insert data to an array**

In [0]:
def groupingData(price, minNight, review, availa):
  data = []

  # Memasukan data price, minNight, review, availa ke-i kedalam list 2 dimensi
  for ab in range(len(price)):
    dataTemp = []
    dataTemp.append(price[ab])
    dataTemp.append(minNight[ab])
    dataTemp.append(review[ab])
    dataTemp.append(availa[ab])
    data.append(dataTemp)
  
  return data

**Insert label to an array**

In [0]:
def groupingLabel(data):
  label = []
  # Memasukan data label list 
  for i in range(len(data)):
    label.append(data[i])

  return label

**Convert Label to Binary**

In [0]:
def convertBinary(data):

  # Convert label menjadi 0 atau 1 atau 2
  lb = preprocessing.LabelEncoder()
  lbData = lb.fit_transform(data)

  return lbData

# **Main**

**Read Dataset**

In [0]:
# Read Dataset
a, b, c, d, e = readDataset()

print("Panjang Data :", len(a), len(b), len(c), len(d), len(e))

Panjang Data : 22552 22552 22552 22552 22552


**Remove Empty Data**

Menghapus data kosong yang ada di kolom

In [0]:
tempPrice = []
tempMinNight = []
tempReview = []
tempAvaila = []
tempRoomType = []
  
# Remove empty data
for i in range(len(b)):
  if a[i] != '' and b[i] != '' and c[i] != '' and d[i] != '' and e[i] != '':
    tempPrice.append(a[i])
    tempMinNight.append(b[i])
    tempReview.append(float(c[i]))
    tempAvaila.append(d[i])
    tempRoomType.append(e[i])

print("Panjang Data :", len(tempPrice), len(tempMinNight), len(tempReview), len(tempAvaila), len(tempRoomType))

Panjang Data : 18638 18638 18638 18638 18638


**Outlier**

In [0]:
no_outliers_price, no_outliers_minNight, no_outliers_review, no_outliers_availa, no_outliers_roomType = outlier(tempPrice, tempMinNight, tempReview, tempAvaila, tempRoomType)

print("Panjang Data :", len(no_outliers_price), len(no_outliers_minNight), len(no_outliers_review), len(no_outliers_availa), len(no_outliers_roomType))

Panjang Data : 16220 16220 16220 16220 16220


**Scaling**

Scaling dilakukan untuk menyamaratakan range dari setiap kolom. Metode yang digunakan adalah **Min-Max Scaler**

In [0]:
newDataPrice = scalingData(no_outliers_price)
newDataReview = scalingData(no_outliers_review)
newDataMinNight = scalingData(no_outliers_minNight)
newDataAvaila = scalingData(no_outliers_availa)

print(newDataPrice)
print(newDataReview)
print(newDataMinNight)
print(newDataAvaila)

# RoomType tidak di scaling karena itu adalah sebuah text

[0.15789473684210525, 0.04473684210526316, 0.23684210526315788, 0.06842105263157895, 0.11052631578947368, 0.47368421052631576, 0.18421052631578946, 0.23684210526315788, 0.11842105263157894, 0.12894736842105264, 0.3394736842105263, 0.18421052631578946, 0.2578947368421053, 0.42105263157894735, 0.17105263157894737, 0.23684210526315788, 0.23684210526315788, 0.18421052631578946, 0.23684210526315788, 0.34210526315789475, 0.2631578947368421, 0.14473684210526316, 0.10526315789473684, 0.1368421052631579, 0.10263157894736842, 0.21052631578947367, 0.21052631578947367, 0.13157894736842105, 0.11315789473684211, 0.10526315789473684, 0.09473684210526316, 0.3131578947368421, 0.23684210526315788, 0.11052631578947368, 0.14736842105263157, 0.13157894736842105, 0.12631578947368421, 0.42105263157894735, 0.23157894736842105, 0.14473684210526316, 0.23684210526315788, 0.14473684210526316, 0.5263157894736842, 0.21052631578947367, 0.12894736842105264, 0.34210526315789475, 0.23684210526315788, 0.3157894736842105

**Split Dataset 80% Train 20% Test**

Memisahkand data menjadi 80% data train dan 20% data test

In [0]:
priceTrain, priceTest, minNightTrain, minNightTest, reviewTrain, reviewTest, availaTrain, availaTest, roomTypeTrain, roomTypeTest = train_test_split(newDataPrice, newDataMinNight, newDataReview, newDataAvaila, no_outliers_roomType, test_size = 0.2, random_state = 0)

print("Panjang Data Train :", len(priceTrain))
print("Panjang Data Label :", len(priceTest))

Panjang Data Train : 12976
Panjang Data Label : 3244


**Convert Label to Integer**

Mengubah label menjadi 0 atau 1 atau 2

In [0]:
lbTrain = convertBinary(roomTypeTrain)
lbTest = convertBinary(roomTypeTest)

print(lbTrain)

[0 0 1 ... 0 1 1]


**Grouping Data and Label**

Mengabungkan data yang terdiri dati price, minNight, review, dan availability kedalam list

In [0]:
dataTrain = groupingData(priceTrain, minNightTrain, reviewTrain, availaTrain)
dataTest = groupingData(priceTest, minNightTest, reviewTest, availaTest)
labelTrain = groupingLabel(lbTrain)
labelTest = groupingLabel(lbTest)

print("Isi Data Train Ke-1  :", dataTrain[0])
print("Isi Data Test Ke-1   :", labelTrain[0])

Isi Data Train Ke-1  : [0.14473684210526316, 0.02197802197802198, 0.2397094430992736, 0.03076923076923077]
Isi Data Test Ke-1   : 0


# Evaluasi

**Klasifikasi**

Metode yang dilakukan adalah **Naive Bayes**

In [0]:
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(dataTrain, labelTrain)

#Predict the response for test dataset
labelPred = gnb.predict(dataTest)

In [0]:
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Akurasi :",metrics.accuracy_score(labelTest, labelPred))

Akurasi : 0.6963625154130703


Akurasi yang didapat sebesar **69,6%**

# Eksperimen

Eksperimen yang dilakuan adalah :

**1. Tidak melakukan Deteksi Outlier**

Read Data

In [0]:
# Read Dataset
a1, b1, c1, d1, e1 = readDataset()

print("Panjang Data :", len(a1), len(b1), len(c1), len(d1), len(e1))

Panjang Data : 22552 22552 22552 22552 22552


Remove Data Empty

In [0]:
tempPrice1 = []
tempMinNight1 = []
tempReview1 = []
tempAvaila1 = []
tempRoomType1 = []
  
# Remove empty data
for i in range(len(b1)):
  if a1[i] != '' and b1[i] != '' and c1[i] != '' and d1[i] != '' and e1[i] != '':
    tempPrice1.append(a1[i])
    tempMinNight1.append(b1[i])
    tempReview1.append(float(c1[i]))
    tempAvaila1.append(d1[i])
    tempRoomType1.append(e1[i])

print("Panjang Data :", len(tempPrice1), len(tempMinNight1), len(tempReview1), len(tempAvaila1), len(tempRoomType1))

Panjang Data : 18638 18638 18638 18638 18638


Scaling

In [0]:
newDataPrice1 = scalingData(tempPrice1)
newDataReview1 = scalingData(tempReview1)
newDataMinNight1 = scalingData(tempMinNight1)
newDataAvaila1 = scalingData(tempAvaila1)

print(newDataPrice1)
print(newDataReview1)
print(newDataMinNight1)
print(newDataAvaila1)

# RoomType tidak di scaling karena itu adalah sebuah text

[0.006666666666666667, 0.001888888888888889, 0.01, 0.0028888888888888888, 0.004666666666666667, 0.02, 0.0077777777777777776, 0.01, 0.005, 0.0054444444444444445, 0.014333333333333333, 0.0077777777777777776, 0.010888888888888889, 0.017777777777777778, 0.007222222222222222, 0.01, 0.01, 0.02188888888888889, 0.0077777777777777776, 0.01, 0.006666666666666667, 0.014444444444444444, 0.011111111111111112, 0.004888888888888889, 0.006888888888888889, 0.006111111111111111, 0.0044444444444444444, 0.002777777777777778, 0.0057777777777777775, 0.004333333333333333, 0.008888888888888889, 0.008888888888888889, 0.005555555555555556, 0.0047777777777777775, 0.0044444444444444444, 0.004, 0.013222222222222222, 0.01, 0.004666666666666667, 0.006222222222222222, 0.005555555555555556, 0.005333333333333333, 0.017777777777777778, 0.009777777777777778, 0.006111111111111111, 0.01, 0.006111111111111111, 0.022222222222222223, 0.008888888888888889, 0.0054444444444444445, 0.006111111111111111, 0.014444444444444444, 0.01

Split Data 80% dan 20%

In [0]:
priceTrain1, priceTest1, minNightTrain1, minNightTest1, reviewTrain1, reviewTest1, availaTrain1, availaTest1, roomTypeTrain1, roomTypeTest1 = train_test_split(newDataPrice1, newDataMinNight1, newDataReview1, newDataAvaila1, tempRoomType1, test_size = 0.2, random_state = 0)

print("Panjang Data Train :", len(priceTrain))
print("Panjang Data Label :", len(priceTest))

Panjang Data Train : 12976
Panjang Data Label : 3244


Convert Label to Integer

In [0]:
lbTrain1 = convertBinary(roomTypeTrain1)
lbTest1 = convertBinary(roomTypeTest1)

print(lbTrain1)

[1 0 1 ... 0 0 1]


Grouping Data

In [0]:
dataTrain1 = groupingData(priceTrain1, minNightTrain1, reviewTrain1, availaTrain1)
dataTest1 = groupingData(priceTest1, minNightTest1, reviewTest1, availaTest1)
labelTrain1 = groupingLabel(lbTrain1)
labelTest1 = groupingLabel(lbTest1)

print("Isi Data Train Ke-1  :", dataTrain1[0])
print("Isi Data Test Ke-1   :", labelTrain1[0])

Isi Data Train Ke-1  : [0.002, 0.0058011602320464095, 0.005455537370430986, 0.09863013698630137]
Isi Data Test Ke-1   : 1


Klasifikasi

In [0]:
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(dataTrain1, labelTrain1)

#Predict the response for test dataset
labelPred1 = gnb.predict(dataTest1)

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Akurasi :",metrics.accuracy_score(labelTest1, labelPred1))

Akurasi : 0.48256437768240346


Akurasi yang didapat sebesar **49,25%**

**2. Tidak melakukan Scaling**

Read Data

In [0]:
# Read Dataset
a2, b2, c2, d2, e2 = readDataset()

print("Panjang Data :", len(a2), len(b2), len(c2), len(d2), len(e2))

Panjang Data : 22552 22552 22552 22552 22552


Remove Empty Data

In [0]:
tempPrice2 = []
tempMinNight2 = []
tempReview2 = []
tempAvaila2 = []
tempRoomType2 = []
  
# Remove empty data
for i in range(len(b)):
  if a2[i] != '' and b2[i] != '' and c2[i] != '' and d2[i] != '' and e2[i] != '':
    tempPrice2.append(a[i])
    tempMinNight2.append(b[i])
    tempReview2.append(float(c[i]))
    tempAvaila2.append(d[i])
    tempRoomType2.append(e[i])

print("Panjang Data :", len(tempPrice2), len(tempMinNight2), len(tempReview2), len(tempAvaila2), len(tempRoomType2))

Panjang Data : 18638 18638 18638 18638 18638


Outlier

In [0]:
no_outliers_price2, no_outliers_minNight2, no_outliers_review2, no_outliers_availa2, no_outliers_roomType2 = outlier(tempPrice2, tempMinNight2, tempReview2, tempAvaila2, tempRoomType2)

print("Panjang Data :", len(no_outliers_price2), len(no_outliers_minNight2), len(no_outliers_review2), len(no_outliers_availa2), len(no_outliers_roomType2))

Panjang Data : 16220 16220 16220 16220 16220


Split Data 80% dan 20%

In [0]:
priceTrain2, priceTest2, minNightTrain2, minNightTest2, reviewTrain2, reviewTest2, availaTrain2, availaTest2, roomTypeTrain2, roomTypeTest2 = train_test_split(no_outliers_price2, no_outliers_minNight2, no_outliers_review2, no_outliers_availa2, no_outliers_roomType2, test_size = 0.2, random_state = 0)

print("Panjang Data Train :", len(priceTrain2))
print("Panjang Data Label :", len(priceTest2))

Panjang Data Train : 12976
Panjang Data Label : 3244


Convert Label

In [0]:
lbTrain2 = convertBinary(roomTypeTrain2)
lbTest2 = convertBinary(roomTypeTest2)

print(lbTrain2)

[0 0 1 ... 0 1 1]


Grouping Data

In [0]:
dataTrain2 = groupingData(priceTrain2, minNightTrain2, reviewTrain2, availaTrain2)
dataTest2 = groupingData(priceTest2, minNightTest2, reviewTest2, availaTest2)
labelTrain2 = groupingLabel(lbTrain2)
labelTest2 = groupingLabel(lbTest2)

print("Isi Data Train Ke-1  :", dataTrain2[0])
print("Isi Data Test Ke-1   :", labelTrain2[0])

Isi Data Train Ke-1  : [55, 3, 1.0, 10]
Isi Data Test Ke-1   : 0


Klasifikasi

In [0]:
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(dataTrain2, labelTrain2)

#Predict the response for test dataset
labelPred2 = gnb.predict(dataTest2)

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Akurasi :",metrics.accuracy_score(labelTest2, labelPred2))

Akurasi : 0.6963625154130703


Akurasi yang didapat sebesar **69,6%**

# Kesimpulan

Dari hasil diatas, dapat disimpulkan bahwa Klasifikasi Room Type Berdasarkan Price, Review, Minimum Night, dan Availability berhasil mendapatkan akurasi sebesar **69,6%** dengan data training dan testing 80-20. Dan akurasi **48,25%** untuk klasifikasi tanpa mendeteksi outlier. Selain itu juga, dengan tidak melakukan scaling pada eksperimen ke-2, akurasi yang didapat tetap sama yaitu **63,6%**. Dapat disimpulkan bahwa, mengunakan persiapan data yang berbeda dapat mempengaruhi hasil akurasi yang didapat. Selain, itu teknik scaling tidak mempengaruhi hasil akurasi.

Terimakasih